# Project Advanced Lane Lines

## Import Packages

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import scipy.stats

## Camera Calibration

The camera is calibrated using the cv2 functions on the following 20 chessboard images.
<figure>
 <img src="output_images/calibration_images.jpg" width="720" alt="Calibration Images" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> These 20 images are used for the camera calibration. </p> 
 </figcaption>
</figure>

In [2]:
def undistortAll(nx=9, ny=6):
    objpoints = []
    imgpoints = []
    shape = []
    objp = np.zeros((nx * ny, 3), np.float32)
    objp[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)

    # Use 20 images for calibration
    fnames = glob.glob('camera_cal/calibration*.jpg')
    for fname in fnames:
        img = plt.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        shape = gray.shape[::-1]
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
        if ret:
            cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
            imgpoints.append(corners)
            objpoints.append(objp)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, shape, None, None)
    return mtx, dist

<figure>
 <img src="output_images/undistorted_chess.jpg" width="720" alt="Undistorted Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> An example of the undistorted chessboard image. </p> 
 </figcaption>
</figure>

## Color Transform
The input image is transformed to a binary image by color transform and gradients.

In [3]:
def thres(img, s_thresh=(170, 255), sx_thresh=(80, 130), h_thresh=(0,180)):
    img = np.copy(img)
    
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    h_binary = np.zeros_like(h_channel)
    h_binary[(h_channel >= h_thresh[0]) & (h_channel <= h_thresh[1])] = 1
    
    # Stack each channel
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    
    hcombined_binary = np.zeros_like(sxbinary)
    hcombined_binary[(h_binary == 1) & ((s_binary == 1) | (sxbinary == 1))] = 1

    return hcombined_binary

## Perspective Transform
The binary image is transformed to a birds-eye-view. The transform matrix is calculated from the test image with the straight lines.
<figure>
 <img src="output_images/transform_straight_lines1.jpg" width="320" alt="Transform Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Straight lane lines used to find the perspective transformation. </p> 
 </figcaption>
</figure>

In [4]:
def getMat():
    src = np.float32([[265,675], [603,444], [677,444], [1015,675]])
    dst = np.float32([[400,720], [400,0], [880,0], [880,720]])
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    return M, Minv

def transform(img, M):
    warped = cv2.warpPerspective(img, M, img.shape[1::-1], flags=cv2.INTER_LINEAR)
    return warped

def getWarped(img, mtx, dist, M):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    binary = thres(undist)
    warped = transform(binary, M)
    return warped

The result of the transforms are shown on the test images.
<figure>
 <img src="output_images/test_warped_0.jpg" width="1024" alt="Transformed Image 1" />
 <img src="output_images/test_warped_1.jpg" width="1024" alt="Transformed Image 2" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Left: Original input image from the camera. Middle: Color transformed binary image after thresholding. Right: Binary image after perspective transform. </p> 
 </figcaption>
</figure>

## Finding lane lines

The lane lines are found by moving boxes around the imgage and finding the active pixels in the box.

The serach begins from the bottom of the image. The initial position of the boxes are determined by the histograms. 

In order to reduce the noise, the histograms are weighted by the guessed x positions for left and right. By default, the guessed x values are taken from the values that are used in the perspective transform of the test image with the straight lines. If the box search is performed in the middle of the video frames, the guess values may be set to the x values from the previous frame.

<figure>
 <img src="output_images/box_search.png" width="480" alt="Lane Line Box Search" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Lane lines found by box search method. </p> 
 </figcaption>
</figure>

In [5]:
def weightedHist(hist, guess_left=400, guess_right=880):
    xlen = hist.shape[0]
    midx = int(xlen / 2)
    guess_width = guess_left / 2
    normx = range(xlen)
    norm_left = scipy.stats.norm.pdf(normx, guess_left, guess_width)
    norm_left /= np.max(norm_left)
    norm_left[midx:] = 1
    norm_right = scipy.stats.norm.pdf(normx, guess_right, guess_width)
    norm_right /= np.max(norm_right)
    norm_right[:midx] = 1
    h2 = hist * norm_right * norm_left
    return h2

def getCurves(lefty, leftx, righty, rightx, y_eval):
    left_curverad = getCurve(lefty, leftx, y_eval)
    right_curverad = getCurve(righty, rightx, y_eval)
    return left_curverad, right_curverad

def getCurve(y, x, y_eval):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/480 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    fit_cr = np.polyfit(y*ym_per_pix, x*xm_per_pix, 2)
    # Calculate the new radii of curvature
    curverad = ((1 + (2*fit_cr[0]*y_eval*ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])
    # Now our radius of curvature is in meters
    return curverad

def boxSearch(warped):
    midx = int(warped.shape[1]/2)
    midy = int(warped.shape[0]/2)
    
    # Take a histogram of the bottom half of the image
    histogram0 = np.sum(warped[midy:,:], axis=0)
    histogram = weightedHist(histogram0)

    # Create an output image to draw on and visualize the result
    out_img = np.dstack((warped, warped, warped))*255
    
    # Find the peak of the left and right halves of the histogram
    leftx_base = np.argmax(histogram[:midx])
    rightx_base = np.argmax(histogram[midx:]) + midx

    # Choose the number of sliding windows
    nwindows = 9
    
    # Set height of windows
    window_height = np.int(warped.shape[0]/nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = warped.shape[0] - (window+1)*window_height
        win_y_high = warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        # Darw the wdindows on the visualization image
        cv2.rectangle(out_img, (win_xleft_low, win_y_low), (win_xleft_high, win_y_high), (0, 255, 0), 2)
        cv2.rectangle(out_img, (win_xright_low, win_y_low), (win_xright_high, win_y_high), (0, 255, 0), 2)

        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
            (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
            (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]

        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        # If uou found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current  = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return lefty, leftx, righty, rightx

## Polynomial based search for lane lines
Once lane lines are found in a frame, we can use the polynomial fits as the reference for the search in the next frame. Following example is a particularly hard one due to the noise from the shadows on the left. However, it finds reasonable lane lines thanks to the information from the previous fits.
<figure>
 <img src="output_images/poly_search.png" width="480" alt="Polynomial Search" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Lane line found by polynomial search method. </p> 
 </figcaption>
</figure>

In [6]:
def polySearch(warped, left_fit, right_fit):
    nonzero = warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = (
    (nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) &
    (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = (
    (nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin))&
    (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return lefty, leftx, righty, rightx

def plotLines(img, warped, mtx, dist, Minv, leftLine, rightLine):
    xm_per_pix = 3.7/480 # meters per pixel in x dimension
    
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0])
    left_fitx = leftLine.avgPlotx
    right_fitx = rightLine.avgPlotx

    undist = cv2.undistort(img, mtx, dist, None, mtx)

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    # Print the curvature radius.
    font = cv2.FONT_HERSHEY_SIMPLEX
    locRad = (50, 50)
    locAvg = (50, 100)
    fontScale = 1
    fontColor = (255, 255, 255)
    lineType = 2
    leftx = leftLine.getX()
    rightx = rightLine.getX()
    offcenter = xm_per_pix * ((leftx + rightx)/2. - warped.shape[1]/2.)
    lr = None
    if offcenter > 0.:
        lr = 'Left'
    else:
        lr = 'Right'
    
    curverad = (leftLine.getAvgCurverad() + rightLine.getAvgCurverad()) / 2.
    
    txt = 'Radius of Curvature = {:5d}m'.format(int(curverad))
    cv2.putText(result, txt, locRad, font, fontScale, fontColor, lineType)
    txtAvg = 'Vehicle is {:.2f}m {} of center'.format(abs(offcenter), lr)
    cv2.putText(result, txtAvg, locAvg, font, fontScale, fontColor, lineType)

    return result

## Test on Video

The above functions are used to find the lane lines in the video file project_video.mp4.

A class `Line` is defined to store the fit results and its history. An exponential moving average is used for its efficiency in calculation and memory usage. The weight `w` is a smoothing constant, or forgetting factor, taking a value between 0 and 1.

For the image processing, a callable class `ProcImg` is used instead of a function. This makes it easier to store a history of previous lanes within the class.

In [7]:
mtx, dist = undistortAll()
M, Minv = getMat()

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

class Line:
    w = 0.5
    pixely = None
    pixelx = None
    avgPlotx = None
    #polynomial coefficients for the most recent fit
    fit = None
    curverad = None
    #polynomial coefficients averaged over the last iterations
    best_fit = None
    def __init__(self, pixely, pixelx, shape):
        # y-dim of the image
        self.ymax = shape[1]
        # x-dim of the image
        self.xmax = shape[0]
        self.ploty = np.linspace(0, shape[0]-1, shape[0])
        self.addNewPixels(pixely, pixelx)
        
    def getX(self):
        return self.avgPlotx[-1]
    
    def getSlope(self, y_eval):
        return 2*self.fit[0]*y_eval + self.fit[1]
    
    def getSlopes(self):
        return np.array([self.getSlope(0), self.getSlope(self.ymax/2), self.getSlope(self.ymax)])
        
    def addNewPixels(self, pixely, pixelx):
        self.pixely = pixely
        self.pixelx = pixelx
        self.fit = np.polyfit(pixely, pixelx, 2)
        self.curverad = getCurve(self.pixely, self.pixelx, self.ymax)
        
    def isGoodCurve(self):
        return self.curverad > 150
        
    def isGoodFit(self):
        return self.isGoodCurve()
    
    def getAvg(self, avg, newVal):
        newAvg = self.w*newVal + (1 - self.w)*avg
        return newAvg
    
    def updateAvgs(self):
        if self.isGoodFit():
            if self.best_fit is None:
                self.best_fit = self.fit
                plotx = self.fit[0]*self.ploty**2 + self.fit[1]*self.ploty + self.fit[2]
                self.avgPlotx = plotx
                self.avgCurverad = self.curverad
            else:
                plotx = self.fit[0]*self.ploty**2 + self.fit[1]*self.ploty + self.fit[2]
                self.avgPlotx = self.getAvg(self.avgPlotx, plotx)
                self.best_fit = np.polyfit(self.ploty, self.avgPlotx, 2)
                
                self.avgCurverad = self.getAvg(self.avgCurverad, self.curverad)
                
    def getCurverad(self):
        return self.curverad
    
    def getAvgCurverad(self):
        return getCurve(self.ploty, self.avgPlotx, self.ymax)

class ProcImg:
    def __init__(self):
        self.leftLine = None
        self.rightLine = None
        self.warped = None
    
    def doFirstSearch(self):
        lefty, leftx, righty, rightx = boxSearch(self.warped)
        self.leftLine = Line(lefty, leftx, self.warped.shape)
        self.rightLine = Line(righty, rightx, self.warped.shape)
    def doBoxSearch(self):
        lefty, leftx, righty, rightx = boxSearch(self.warped)
        self.leftLine.addNewPixels(lefty, leftx)
        self.rightLine.addNewPixels(righty, rightx)
    def doPolySearch(self):
        lefty, leftx, righty, rightx = polySearch(self.warped, \
                    self.leftLine.best_fit, self.rightLine.fit)
        self.leftLine.addNewPixels(lefty, leftx)
        self.rightLine.addNewPixels(righty, rightx)
        
    def goodAngle(self, left_slope, right_slope):
        left_angle = np.arctan(left_slope)
        right_angle = np.arctan(right_slope)
        diff_angle = np.absolute(left_angle - right_angle)
        max_angle = np.array([0.1, 0.2, 0.3])
        return all(diff_angle < max_angle)
        
    def linesCompatible(self):
        if self.leftLine is None or self.rightLine is None:
            return True
        left_slopes = self.leftLine.getSlopes()
        right_slopes = self.rightLine.getSlopes()
        return self.goodAngle(left_slopes, right_slopes)

    def __call__(self, img):
        self.warped = getWarped(img, mtx, dist, M)
        result = img
    
        if self.leftLine is None:
            self.doFirstSearch()
            self.leftLine.updateAvgs()
            self.rightLine.updateAvgs()
        else:
            self.doPolySearch()
            if not (self.linesCompatible() and self.leftLine.isGoodFit() and self.rightLine.isGoodFit()):
                self.doBoxSearch()
                
            if self.linesCompatible():
                self.leftLine.updateAvgs()
                self.rightLine.updateAvgs()
            
        if self.leftLine.best_fit is not None and self.rightLine.best_fit is not None:
            result = plotLines(img, self.warped, mtx, dist, Minv,
                          self.leftLine, self.rightLine)
        return result

In [8]:
pimg = ProcImg()
project_output = 'output_images/project.mp4'
clip1 = VideoFileClip("project_video.mp4")
project_clip = clip1.fl_image(pimg)
%time project_clip.write_videofile(project_output, audio=False)

[MoviePy] >>>> Building video output_images/project.mp4
[MoviePy] Writing video output_images/project.mp4


100%|█████████▉| 1260/1261 [04:06<00:00,  5.33it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/project.mp4 

CPU times: user 5min 22s, sys: 16.7 s, total: 5min 39s
Wall time: 4min 7s


In [9]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_output))